# 📈 Moving Average Strategy Testing

Welcome to Notebook 02! In this notebook, you'll learn how to:

1. **Implement Moving Average crossover strategies**
2. **Backtest strategies** on historical data
3. **Optimize parameters** for better performance
4. **Visualize strategy performance** with equity curves

Let's dive into the world of Moving Average strategies!

## Step 1: Import Libraries and Load Data

First, let's import everything we need and load our data.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import our custom modules
import sys
sys.path.append('../')
from strategies.base_strategy import SimpleMovingAverageStrategy
from utils.data_loader import load_sample_data, load_mt5_csv, validate_data
from utils.visualization import TradingVisualizer

# Set up plotting
plt.style.use('seaborn-v0_8')
import plotly.io as pio
pio.renderers.default = "notebook"

print("✅ All libraries imported successfully!")

# Load data (use sample data or your MT5 data)
print("📁 Loading data...")
data = load_sample_data()  # You can replace this with your MT5 data
# data = load_mt5_csv("../data/raw/your_file.csv")  # Uncomment to use your data

print(f"📊 Data loaded: {data.shape[0]} rows, {data.shape[1]} columns")
print(f"📅 Date range: {data.index.min()} to {data.index.max()}")

## Step 2: Understanding Moving Averages

Before we implement the strategy, let's understand what Moving Averages are:

In [ ]:
# Calculate and visualize different moving averages
print("📊 Calculating Moving Averages...")

# Calculate different moving averages
ma_10 = data['close'].rolling(window=10).mean()
ma_20 = data['close'].rolling(window=20).mean()
ma_50 = data['close'].rolling(window=50).mean()
ma_100 = data['close'].rolling(window=100).mean()

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Moving Averages Analysis', fontsize=16)

# Plot 1: Short-term MAs
axes[0, 0].plot(data.index, data['close'], label='Close Price', color='blue', alpha=0.7)
axes[0, 0].plot(data.index, ma_10, label='MA 10', color='orange', linewidth=2)
axes[0, 0].plot(data.index, ma_20, label='MA 20', color='red', linewidth=2)
axes[0, 0].set_title('Short-term Moving Averages')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Long-term MAs
axes[0, 1].plot(data.index, data['close'], label='Close Price', color='blue', alpha=0.7)
axes[0, 1].plot(data.index, ma_50, label='MA 50', color='green', linewidth=2)
axes[0, 1].plot(data.index, ma_100, label='MA 100', color='purple', linewidth=2)
axes[0, 1].set_title('Long-term Moving Averages')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: MA Crossover example
axes[1, 0].plot(data.index, ma_20, label='MA 20', color='red', linewidth=2)
axes[1, 0].plot(data.index, ma_50, label='MA 50', color='blue', linewidth=2)

# Highlight crossovers
crossover_points = []
for i in range(1, len(data)):
    if (ma_20.iloc[i-1] < ma_50.iloc[i-1]) and (ma_20.iloc[i] > ma_50.iloc[i]):
        crossover_points.append((data.index[i], ma_20.iloc[i]))
    elif (ma_20.iloc[i-1] > ma_50.iloc[i-1]) and (ma_20.iloc[i] < ma_50.iloc[i]):
        crossover_points.append((data.index[i], ma_20.iloc[i]))

if crossover_points:
    crossover_dates, crossover_prices = zip(*crossover_points)
    axes[1, 0].scatter(crossover_dates, crossover_prices, color='black', s=50, zorder=5, label='Crossovers')

axes[1, 0].set_title('MA Crossover Points')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Price with MA signals
axes[1, 1].plot(data.index, data['close'], label='Close Price', color='blue', linewidth=1)
axes[1, 1].plot(data.index, ma_20, label='MA 20', color='red', linewidth=2, alpha=0.7)
axes[1, 1].plot(data.index, ma_50, label='MA 50', color='green', linewidth=2, alpha=0.7)

# Add buy/sell signals
buy_signals = data[(ma_20 > ma_50) & (ma_20.shift(1) <= ma_50.shift(1))]
sell_signals = data[(ma_20 < ma_50) & (ma_20.shift(1) >= ma_50.shift(1))]

if not buy_signals.empty:
    axes[1, 1].scatter(buy_signals.index, buy_signals['close'], color='green', marker='^', s=100, label='Buy Signal')

if not sell_signals.empty:
    axes[1, 1].scatter(sell_signals.index, sell_signals['close'], color='red', marker='v', s=100, label='Sell Signal')

axes[1, 1].set_title('Price with MA Crossover Signals')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ Moving averages calculated successfully!")
print(f"📈 Found {len(buy_signals)} buy signals and {len(sell_signals)} sell signals")

## Step 3: Implement Your First MA Strategy

Now let's create and test our first Moving Average crossover strategy!

In [ ]:
# Create a simple MA crossover strategy
print("🚀 Creating Moving Average Crossover Strategy...")

# Strategy parameters
fast_period = 10
slow_period = 30

print(f"📊 Strategy Parameters:")
print(f"   Fast MA Period: {fast_period}")
print(f"   Slow MA Period: {slow_period}")

# Create strategy instance
strategy = SimpleMovingAverageStrategy(fast_period=fast_period, slow_period=slow_period)

print(f"✅ Strategy created: {strategy.name}")
print(f"⚙️  Parameters: {strategy.get_parameters()}")

In [ ]:
# Generate trading signals
print("📊 Generating trading signals...")

signals = strategy.generate_signals(data)

# Analyze signals
buy_signals = sum(signals == 1)
sell_signals = sum(signals == -1)
hold_signals = sum(signals == 0)

print(f"✅ Signals generated successfully!")
print(f"📈 Buy signals: {buy_signals}")
print(f"📉 Sell signals: {sell_signals}")
print(f"⏸️  Hold signals: {hold_signals}")
print(f"📊 Signal distribution: {buy_signals/len(signals)*100:.1f}% buy, {sell_signals/len(signals)*100:.1f}% sell")

## Step 4: Backtest the Strategy

Now let's backtest our strategy to see how it would have performed!

In [ ]:
# Backtest the strategy
print("💰 Backtesting Strategy...")

# Backtest parameters
initial_capital = 10000
lot_size = 1.0
stop_loss = 0.001  # 10 pips
take_profit = 0.003  # 30 pips
commission = 0.00005  # 0.5 pips

print(f"📊 Backtest Parameters:")
print(f"   Initial Capital: ${initial_capital:,}")
print(f"   Lot Size: {lot_size}")
print(f"   Stop Loss: {stop_loss} ({stop_loss*10000:.0f} pips)")
print(f"   Take Profit: {take_profit} ({take_profit*10000:.0f} pips)")
print(f"   Commission: {commission} ({commission*10000:.1f} pips)")

# Run backtest
results = strategy.backtest(
    data=data,
    initial_capital=initial_capital,
    lot_size=lot_size,
    stop_loss=stop_loss,
    take_profit=take_profit,
    commission=commission
)

print(f"\n✅ Backtest completed!")

In [ ]:
# Display backtest results
print("📈 BACKTEST RESULTS")
print("=" * 50)

if 'error' in results:
    print(f"❌ Error: {results['error']}")
else:
    # Performance metrics
    print(f"💰 FINANCIAL PERFORMANCE:")
    print(f"   Initial Capital: ${results['initial_capital']:,.2f}")
    print(f"   Final Capital: ${results['final_capital']:,.2f}")
    print(f"   Total Return: {results['total_return']:.2f}%")
    
    print(f"\n📊 RISK METRICS:")
    print(f"   Sharpe Ratio: {results['sharpe_ratio']:.2f}")
    print(f"   Maximum Drawdown: {results['max_drawdown']:.2f}%")
    
    print(f"\n🎯 TRADE STATISTICS:")
    print(f"   Total Trades: {results['total_trades']}")
    print(f"   Winning Trades: {results['winning_trades']}")
    print(f"   Losing Trades: {results['losing_trades']}")
    print(f"   Win Rate: {results['win_rate']*100:.1f}%")
    print(f"   Profit Factor: {results['profit_factor']:.2f}")
    
    print(f"\n💰 AVERAGE TRADES:")
    print(f"   Average Win: ${results['avg_win']:.2f}")
    print(f"   Average Loss: ${results['avg_loss']:.2f}")

## Step 5: Visualize Strategy Performance

Let's create beautiful visualizations to understand our strategy's performance!

In [ ]:
# Create visualizations
print("🎨 Creating Strategy Visualizations...")

# Initialize visualizer
viz = TradingVisualizer()

# 1. Price chart with signals
indicators = {
    f'SMA_{fast_period}': data['close'].rolling(window=fast_period).mean(),
    f'SMA_{slow_period}': data['close'].rolling(window=slow_period).mean()
}

fig1 = viz.create_price_chart(
    data=data,
    indicators=indicators,
    signals=signals,
    title=f"EURUSD with {fast_period}/{slow_period} MA Crossover Strategy"
)
fig1.show()

print("✅ Price chart with signals created!")

In [ ]:
# 2. Equity curve
fig2 = viz.create_equity_curve(
    equity_curve=results['equity_curve'],
    trades=results['trades'],
    title="Strategy Equity Curve"
)
fig2.show()

print("✅ Equity curve created!")

In [ ]:
# 3. Performance dashboard
fig3 = viz.create_performance_dashboard(results)
fig3.show()

print("✅ Performance dashboard created!")

In [ ]:
# 4. Trade analysis
fig4 = viz.create_trade_analysis(results['trades'])
fig4.show()

print("✅ Trade analysis created!")

## Step 6: Parameter Optimization

Let's find the best parameters for our strategy by testing different combinations!

In [ ]:
# Parameter optimization
print("🔍 Optimizing Strategy Parameters...")

# Define parameter ranges
fast_periods = [5, 10, 15, 20]
slow_periods = [20, 30, 50, 100]

print(f"📊 Testing {len(fast_periods) * len(slow_periods)} parameter combinations...")

# Store results
optimization_results = []

# Test all combinations
for fast in fast_periods:
    for slow in slow_periods:
        if fast >= slow:  # Skip invalid combinations
            continue
            
        # Create strategy with current parameters
        test_strategy = SimpleMovingAverageStrategy(fast_period=fast, slow_period=slow)
        
        # Run backtest
        test_results = test_strategy.backtest(
            data=data,
            initial_capital=initial_capital,
            lot_size=lot_size,
            stop_loss=stop_loss,
            take_profit=take_profit,
            commission=commission
        )
        
        # Store results
        optimization_results.append({
            'fast_period': fast,
            'slow_period': slow,
            'total_return': test_results.get('total_return', 0),
            'sharpe_ratio': test_results.get('sharpe_ratio', 0),
            'win_rate': test_results.get('win_rate', 0),
            'max_drawdown': test_results.get('max_drawdown', 0),
            'profit_factor': test_results.get('profit_factor', 0)
        })

# Convert to DataFrame for analysis
opt_df = pd.DataFrame(optimization_results)
print(f"✅ Optimization completed! Tested {len(opt_df)} combinations.")

In [ ]:
# Analyze optimization results
print("📊 OPTIMIZATION RESULTS")
print("=" * 50)

# Sort by different metrics
best_return = opt_df.loc[opt_df['total_return'].idxmax()]
best_sharpe = opt_df.loc[opt_df['sharpe_ratio'].idxmax()]
best_win_rate = opt_df.loc[opt_df['win_rate'].idxmax()]
best_profit_factor = opt_df.loc[opt_df['profit_factor'].idxmax()]

print(f"🏆 BEST TOTAL RETURN:")
print(f"   Parameters: MA {best_return['fast_period']}/{best_return['slow_period']}")
print(f"   Return: {best_return['total_return']:.2f}%")
print(f"   Sharpe: {best_return['sharpe_ratio']:.2f}")
print(f"   Win Rate: {best_return['win_rate']*100:.1f}%")

print(f"\n🏆 BEST SHARPE RATIO:")
print(f"   Parameters: MA {best_sharpe['fast_period']}/{best_sharpe['slow_period']}")
print(f"   Return: {best_sharpe['total_return']:.2f}%")
print(f"   Sharpe: {best_sharpe['sharpe_ratio']:.2f}")
print(f"   Win Rate: {best_sharpe['win_rate']*100:.1f}%")

print(f"\n🏆 BEST WIN RATE:")
print(f"   Parameters: MA {best_win_rate['fast_period']}/{best_win_rate['slow_period']}")
print(f"   Return: {best_win_rate['total_return']:.2f}%")
print(f"   Sharpe: {best_win_rate['sharpe_ratio']:.2f}")
print(f"   Win Rate: {best_win_rate['win_rate']*100:.1f}%")

print(f"\n🏆 BEST PROFIT FACTOR:")
print(f"   Parameters: MA {best_profit_factor['fast_period']}/{best_profit_factor['slow_period']}")
print(f"   Return: {best_profit_factor['total_return']:.2f}%")
print(f"   Sharpe: {best_profit_factor['sharpe_ratio']:.2f}")
print(f"   Profit Factor: {best_profit_factor['profit_factor']:.2f}")

In [ ]:
# Visualize optimization results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Parameter Optimization Results', fontsize=16)

# 1. Total Return heatmap
pivot_return = opt_df.pivot(index='slow_period', columns='fast_period', values='total_return')
sns.heatmap(pivot_return, annot=True, fmt='.1f', cmap='RdYlGn', center=0, ax=axes[0,0])
axes[0,0].set_title('Total Return (%)')

# 2. Sharpe Ratio heatmap
pivot_sharpe = opt_df.pivot(index='slow_period', columns='fast_period', values='sharpe_ratio')
sns.heatmap(pivot_sharpe, annot=True, fmt='.2f', cmap='viridis', ax=axes[0,1])
axes[0,1].set_title('Sharpe Ratio')

# 3. Win Rate heatmap
pivot_win = opt_df.pivot(index='slow_period', columns='fast_period', values='win_rate') * 100
sns.heatmap(pivot_win, annot=True, fmt='.1f', cmap='Blues', ax=axes[1,0])
axes[1,0].set_title('Win Rate (%)')

# 4. Profit Factor heatmap
pivot_pf = opt_df.pivot(index='slow_period', columns='fast_period', values='profit_factor')
sns.heatmap(pivot_pf, annot=True, fmt='.2f', cmap='Greens', ax=axes[1,1])
axes[1,1].set_title('Profit Factor')

plt.tight_layout()
plt.show()

print("✅ Optimization heatmap created!")

## Step 7: Test Your Optimized Strategy

Let's test the best strategy we found!

In [ ]:
# Test the best strategy
print("🚀 Testing Optimized Strategy...")

# Choose the best strategy (you can change this based on your preference)
# Let's use the one with the highest total return
best_params = opt_df.loc[opt_df['total_return'].idxmax()]

print(f"📊 Using optimized parameters: MA {best_params['fast_period']}/{best_params['slow_period']}")

# Create optimized strategy
optimized_strategy = SimpleMovingAverageStrategy(
    fast_period=int(best_params['fast_period']),
    slow_period=int(best_params['slow_period'])
)

# Run backtest with optimized parameters
optimized_results = optimized_strategy.backtest(
    data=data,
    initial_capital=initial_capital,
    lot_size=lot_size,
    stop_loss=stop_loss,
    take_profit=take_profit,
    commission=commission
)

print(f"\n✅ Optimized strategy backtest completed!")

In [ ]:
# Compare original vs optimized
print("📊 COMPARISON: Original vs Optimized")
print("=" * 50)

print(f"💰 FINANCIAL PERFORMANCE:")
print(f"   Original Strategy:")
print(f"     Total Return: {results['total_return']:.2f}%")
print(f"     Final Capital: ${results['final_capital']:,.2f}")
print(f"   Optimized Strategy:")
print(f"     Total Return: {optimized_results['total_return']:.2f}%")
print(f"     Final Capital: ${optimized_results['final_capital']:,.2f}")

improvement = optimized_results['total_return'] - results['total_return']
print(f"   Improvement: {improvement:+.2f}%")

print(f"\n📊 RISK METRICS:")
print(f"   Original Sharpe: {results['sharpe_ratio']:.2f}")
print(f"   Optimized Sharpe: {optimized_results['sharpe_ratio']:.2f}")
print(f"   Original Max DD: {results['max_drawdown']:.2f}%")
print(f"   Optimized Max DD: {optimized_results['max_drawdown']:.2f}%")

print(f"\n🎯 TRADE STATISTICS:")
print(f"   Original Win Rate: {results['win_rate']*100:.1f}%")
print(f"   Optimized Win Rate: {optimized_results['win_rate']*100:.1f}%")
print(f"   Original Profit Factor: {results['profit_factor']:.2f}")
print(f"   Optimized Profit Factor: {optimized_results['profit_factor']:.2f}")

## Step 8: Strategy Robustness Testing

Let's test how our strategy performs under different market conditions!

In [ ]:
# Test strategy on different time periods
print("🧪 Testing Strategy Robustness...")

# Split data into different periods
total_days = len(data)
period_1 = data.iloc[:total_days//3]      # First third
period_2 = data.iloc[total_days//3:2*total_days//3]  # Middle third
period_3 = data.iloc[2*total_days//3:]     # Last third

periods = [
    ("First Period", period_1),
    ("Middle Period", period_2),
    ("Last Period", period_3)
]

# Test strategy on each period
robustness_results = []

for period_name, period_data in periods:
    if len(period_data) < max(best_params['fast_period'], best_params['slow_period']):
        continue  # Skip if not enough data
        
    period_result = optimized_strategy.backtest(
        data=period_data,
        initial_capital=initial_capital,
        lot_size=lot_size,
        stop_loss=stop_loss,
        take_profit=take_profit,
        commission=commission
    )
    
    robustness_results.append({
        'period': period_name,
        'return': period_result['total_return'],
        'sharpe': period_result['sharpe_ratio'],
        'win_rate': period_result['win_rate'],
        'max_dd': period_result['max_drawdown']
    })
    
    print(f"✅ {period_name}: {period_result['total_return']:.2f}% return")

print(f"\n✅ Robustness testing completed!")

In [ ]:
# Visualize robustness results
robust_df = pd.DataFrame(robustness_results)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Strategy Robustness Testing', fontsize=16)

# 1. Returns by period
axes[0, 0].bar(robust_df['period'], robust_df['return'], color=['blue', 'orange', 'green'])
axes[0, 0].set_title('Returns by Period')
axes[0, 0].set_ylabel('Return (%)')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Sharpe ratio by period
axes[0, 1].bar(robust_df['period'], robust_df['sharpe'], color=['blue', 'orange', 'green'])
axes[0, 1].set_title('Sharpe Ratio by Period')
axes[0, 1].set_ylabel('Sharpe Ratio')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Win rate by period
axes[1, 0].bar(robust_df['period'], robust_df['win_rate']*100, color=['blue', 'orange', 'green'])
axes[1, 0].set_title('Win Rate by Period')
axes[1, 0].set_ylabel('Win Rate (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Max drawdown by period
axes[1, 1].bar(robust_df['period'], robust_df['max_dd'], color=['blue', 'orange', 'green'])
axes[1, 1].set_title('Max Drawdown by Period')
axes[1, 1].set_ylabel('Drawdown (%)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Calculate consistency metrics
returns_std = robust_df['return'].std()
sharpe_std = robust_df['sharpe'].std()
win_rate_std = robust_df['win_rate'].std()

print(f"📊 CONSISTENCY METRICS:")
print(f"   Return Standard Deviation: {returns_std:.2f}%")
print(f"   Sharpe Ratio Standard Deviation: {sharpe_std:.2f}")
print(f"   Win Rate Standard Deviation: {win_rate_std*100:.1f}%")

if returns_std < 5:  # Less than 5% variation is good
    print(f"✅ Strategy shows good consistency across periods!")
else:
    print(f"⚠️  Strategy performance varies significantly across periods.")

## 🎯 Summary and Next Steps

Congratulations! You've successfully:

✅ **Implemented** a Moving Average crossover strategy
✅ **Backtested** the strategy on historical data
✅ **Optimized** parameters for better performance
✅ **Visualized** strategy performance with equity curves
✅ **Tested** strategy robustness across different periods

### Key Learnings:

- **Moving Averages** are trend-following indicators that smooth price data
- **Crossover strategies** generate signals when fast MA crosses slow MA
- **Parameter optimization** can significantly improve strategy performance
- **Backtesting** helps evaluate strategy performance before live trading
- **Robustness testing** ensures strategy works in different market conditions

### What's Next?

1. **Move to Notebook 03**: Performance analysis and metrics
2. **Experiment with different strategies**: Try EMA, MACD, RSI
3. **Add more risk management**: Position sizing, trailing stops
4. **Test on different assets**: BTCUSD, XAUUSD, stocks
5. **Create your own strategy**: Combine multiple indicators

### Tips for Success:

- **Start simple**: Master basic strategies before adding complexity
- **Always validate**: Test strategies on out-of-sample data
- **Risk management**: Never risk more than you can afford to lose
- **Keep learning**: The markets are always evolving

You're doing great! Keep experimenting and learning. 🚀